In [ ]:
import os
from flask import Flask, render_template, send_from_directory, url_for, g, request
from werkzeug.utils import secure_filename
import sqlite3
import datetime
from markupsafe import escape

app = Flask(__name__)

@app.route('/')
def index():
    return "Index page"

@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('child.html', name=name)

if __name__ == '__main__':
    app.run()

In [7]:
import os
from flask import Flask, render_template, send_from_directory, url_for, g, request, flash, redirect
from werkzeug.utils import secure_filename
import sqlite3
import datetime
from markupsafe import escape

UPLOAD_FOLDER = "uploads"



app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# @app.route('/')
# def index():
#     return redirect('upload')

@app.route('/', methods=['GET','POST'])
def upload():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        
        file = request.files['file']

        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        
        if file : 
            securename = secure_filename(file.filename)
            path = os.path.join(app.config['UPLOAD_FOLDER'], securename)
            conn = sqlite3.connect("file storage.db")
            c = conn.cursor()
            c.execute("INSERT OR IGNORE INTO files(securename, actualname, path) VALUES (?,?,?);", (securename, file.filename, path))
            conn.commit()
            conn.close()
            file.save(path)
            return(redirect("/download"))
    return render_template('index.html')
    

@app.route("/download/<name>")
def download_file(name):

    conn = sqlite3.connect("file storage.db")
    c = conn.cursor()
    c.execute("SELECT securename, path from files WHERE securename = ?", (name,))
    result = c.fetchone()
    if result:
        securename, path = result
        if os.path.exists(path):
            return send_from_directory(UPLOAD_FOLDER, securename)
        else:
            c.execute("DELETE FROM files WHERE securename = ?",(name,))
            conn.commit()
            conn.close()
            return "File not found. Corresponding record deleted from database"
    else:
        return "File not found"


    
@app.route("/download")
def list_downloads():
    conn = sqlite3.connect("file storage.db")
    c = conn.cursor()
    c.execute("SELECT actualname, securename FROM files")
    files = c.fetchall()
    return render_template('list.html', files=files)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Apr/2023 11:22:55] "GET /download HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 11:22:55] "GET /download/58480979cef1014c0b5e4901.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:22:55] "GET /download/alma_car.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:22:55] "GET /download/AirglowRings_Wang_1080.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:22:55] "GET /download/2_jiongnima.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:22:55] "GET /download/6._EATEOT_-_Stage_6_Cover.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:23:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 11:23:02] "GET /download HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 11:23:02] "GET /download/58480979cef1014c0b5e4901.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:23:02] "GET /download/alma_car.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Apr/2023 11:23:02] "GET /download/AirglowRings_Wang_1080.jpg HTTP/1.1" 304 -
127.0.0.1 - - [25/A

In [2]:
# open the input file
with open('input_file.txt', 'r') as input_file:
    # read the input file as a single string
    input_string = input_file.read()

    # split the input string into 4000 character chunks
    chunks = [input_string[i:i+4000] for i in range(0, len(input_string), 4000)]

    # write each chunk to a separate output file
    for i, chunk in enumerate(chunks):
        output_file_name = f"output_file_{i+1}.txt"
        with open(output_file_name, 'w') as output_file:
            output_file.write(chunk)
